# Have several agents collaborate in a multi-agent hierarchy 🤖🤝🤖
_Authored by: [Aymeric Roucher](https://huggingface.co/m-ric)_

> This tutorial is advanced. You should have notions from [this other cookbook](agents) first!

In this notebook we will make a **multi-agent web browser: an agentic system with several agents collaborating to solve problems using the web!**

Let's set up this system. 

Run the line below to install required dependancies:

In [ ]:
!pip install markdownify duckduckgo-search "git+https://github.com/huggingface/transformers.git#egg=transformers[agents]"

We first create the agent. We used a `ReactCodeAgent` (read the [documentation](https://huggingface.co/docs/transformers/en/agents) to learn more about types of agents), so we do not even need to give it any tools: it can directly run its code.

We simply make sure to let it use data science-related libraries by passing these in `additional_authorized_imports`: `["numpy", "pandas", "matplotlib.pyplot", "seaborn"]`.

In general when passing libraries in `additional_authorized_imports`, make sure they are installed on your local environment, since the python interpreter can only use libraries installed on your environment.

⚙ Our agent will be powered by [meta-llama/Meta-Llama-3.1-70B-Instruct](https://huggingface.co/meta-llama/Meta-Llama-3.1-70B-Instruct) using `HfApiEngine` class that uses HF's Inference API: the Inference API allows to quickly and easily run any OS model.

In [8]:
from huggingface_hub import login
from dotenv import load_dotenv
import os

load_dotenv()

login(os.getenv("HUGGINGFACEHUB_API_TOKEN"))

model = "meta-llama/Meta-Llama-3.1-70B-Instruct"

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /Users/aymeric/.cache/huggingface/token
Login successful


### 🔍 Create a web search tool

For web browsing, we can already use our preexisting `DuckDuckGoSearchTool` tool to provide a Google search equivalent.

But then we will need to be able to peak into page.

So for this, let's create a new tool using `markdownify`.

In [9]:
from transformers import Tool
import requests
from markdownify import markdownify as md
from requests.exceptions import RequestException
import re


class VisitPageTool(Tool):
    name = "wisit_webpage"
    description = "Visits a wbepage at the given url and returns its content as a markdown string."
    inputs = {
        "url": {
            "type": "text",
            "description": "The url of the webpage to visit.",
        }
    }
    output_type = "text"

    def forward(self, url: str) -> str:
        try:
            # Send a GET request to the URL
            response = requests.get(url)
            response.raise_for_status()  # Raise an exception for bad status codes

            # Convert the HTML content to Markdown
            markdown_content = md(response.text).strip()

            # Remove multiple line breaks
            markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)

            return markdown_content

        except RequestException as e:
            return f"Error fetching the webpage: {str(e)}"
        except Exception as e:
            return f"An unexpected error occurred: {str(e)}"

Ok, now let's initialize and test our tool!

In [10]:
visit_page_tool = VisitPageTool()

print(visit_page_tool("https://en.wikipedia.org/wiki/Hugging_Face"))

Hugging Face \- Wikipedia

[Jump to content](#bodyContent)

Main menu

Main menu
move to sidebar
hide

 Navigation
 

* [Main page](/wiki/Main_Page "Visit the main page [z]")
* [Contents](/wiki/Wikipedia:Contents "Guides to browsing Wikipedia")
* [Current events](/wiki/Portal:Current_events "Articles related to current events")
* [Random article](/wiki/Special:Random "Visit a randomly selected article [x]")
* [About Wikipedia](/wiki/Wikipedia:About "Learn about Wikipedia and how it works")
* [Contact us](//en.wikipedia.org/wiki/Wikipedia:Contact_us "How to contact Wikipedia")
* [Donate](https://donate.wikimedia.org/wiki/Special:FundraiserRedirector?utm_source=donate&utm_medium=sidebar&utm_campaign=C13_en.wikipedia.org&uselang=en "Support us by donating to the Wikimedia Foundation")

 Contribute
 

* [Help](/wiki/Help:Contents "Guidance on how to use and edit Wikipedia")
* [Learn to edit](/wiki/Help:Introduction "Learn how to edit Wikipedia")
* [Community portal](/wiki/Wikipedia:Communi

## Build our multi-agent system 🤖🤝🤖

First, we create the web agent, with our two web browsing tools : `search` and `visit_page`.

Which configuration to choose for this one?
- We make it a `ReactJsonAgent`, since web browsing is a single-timeline task that does not require parallel tool calls, so JSON tool calling works well for that.
- Also, since sometimes web search requires exploring many pages before finding the correct answer, we prefer to increase the number of `max_iterations`

In [11]:
from transformers.agents import (
    ReactCodeAgent,
    ReactJsonAgent,
    HfApiEngine,
    ManagedAgent,
)
from transformers.agents.search import DuckDuckGoSearchTool

llm_engine = HfApiEngine(model)

web_agent = ReactJsonAgent(
    tools=[DuckDuckGoSearchTool(), VisitPageTool()],
    llm_engine=llm_engine,
    max_iterations=10,
)

In [12]:
managed_web_agent = ManagedAgent(
    agent=web_agent,
    name="search",
    description="Runs web searches for you. Give it your query as an argument.",
)

Finally we create a manager agent, and upon initialization we pass our managed agent to it in its `managed_agents` argument.

Since this agent is the one charged with the planning and thinking, a `ReactCodeAgent` will be the best choice.

In [13]:
manager_agent = ReactCodeAgent(
    tools=[],
    llm_engine=llm_engine,
    managed_agents=[managed_web_agent],
    additional_authorized_imports=["datetime", "time"],
)

That's all! Now let's run our system! We select a question that requires some calculation and 

In [14]:
manager_agent.run("How many years ago was the series A of startup Hugging Face?")

======== New task ========
How many years ago was the series A of startup Hugging Face?
=== Agent thoughts:
Thought: To determine how many years ago the series A of startup Hugging Face occurred, I first need to find out when the series A took place. I'll use the `search` tool to find this information.
>>> Agent is executing the code below:
series_a_date = search(query="Hugging Face series A date")
print("Series A date:", series_a_date)
====
Code execution failed due to the following error:
ManagedAgent.__call__() missing 1 required positional argument: 'request'
Traceback (most recent call last):
  File "/Users/aymeric/Documents/Code/original_transformers/transformers/src/transformers/agents/agents.py", line 1139, in step
    result = self.python_evaluator(
             ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/aymeric/Documents/Code/original_transformers/transformers/src/transformers/agents/python_interpreter.py", line 895, in evaluate_python_code
    result = evaluate_ast(node, state, s

"Based on the available information, I couldn't find the exact date or year of Hugging Face's series A funding. However, I found that Hugging Face raised $40 million in November 2021, but it's unclear if this was the series A funding.\n\nTo provide a more accurate answer, I'll look for alternative information sources. According to Crunchbase, Hugging Face's series A funding was in 2019, where they raised $30 million.\n\nAssuming this information is correct, and considering the current year (2024), I'll provide an estimated answer:\n\nThe series A of startup Hugging Face was approximately 5 years ago."

The test predictions that the agent output above, once submitted to Kaggle, score **0.78229**, which is #2824 out of 17,360, and better than what I had painfully achieved when first trying the challenge years ago.

Your result will vary, but anyway I find it very impressive to achieve this with an agent in a few seconds.

🚀 The above is just a naive attempt with agent data analyst: it can certainly be improved a lot to fit your use case better!